## Tutorial 8: Building an earthquake catalogue using PyOcto

In this demonstration we will show how to automatically build an earthquake catalogue using SeisBench and the PyOcto associator with OBS data to create an earthquake catalog from raw waveforms. We will use example data from the [YH](https://ds.iris.edu/gmap/#network=YH&starttime=2014-01-01&endtime=2015-06-01&planet=earth) network from the HOBITSS deployment in 2014-2015.

First, we import the seisbench and pyocto packages, as well as other required packages and modules to import data and make plots.

In [ ]:
# SeisBench models
import seisbench.models as sbm

# PyOcto associator
import pyocto

# ObsPy classes
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, Stream

# Other ancillary packages
from pyproj import CRS, Transformer
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt

### Download data

We download waveform data for 60 minutes from the YH network offshore northern New Zealand (HOBITSS deployment). We use October 2, 2014, which corresponds to a magnitude [4.6 earthquake](https://www.geonet.org.nz/earthquake/technical/2014p742742) in the vicinity of the HOBITSS deployment. Therefore, we expect to detect low-magnitude aftershocks in this region.

In [ ]:
# Initialize the FDSN client
client = Client()

# Start time in UTC at the mainshock
t0 = UTCDateTime("2014-10-02T19:33:00")

# End time, 60 minutes later
t1 = t0 + 60 * 60

# Get waveform data
stream = client.get_waveforms(network="YH", station="*", location="*", channel="?H?", starttime=t0, endtime=t1)

# Get station metadata
inv = client.get_stations(network="YH", station="*", location="*", channel="?H?", starttime=t0, endtime=t1)

### Annotate waveforms

For this example, we use OBSTransformer model. However, in principle any picker could be used for obtaining the picks with only minimal changes. 

> Warning: This will take some time and requires sufficient main memory. If you are short on memory, reduce the study time in the cell before.

In [ ]:
# Load the pre-trained OBSTransformer model weights
#picker = sbm.OBSTransformer.from_pretrained('obst2024')
picker = sbm.PickBlue()

# We tuned the thresholds a bit - Feel free to play around with these values
picks = picker.classify(stream, batch_size=256, P_threshold=0.075, S_threshold=0.1).picks

# Output number of P and S picks
Counter([p.phase for p in picks])

### PyOcto association

The following cells configure the pyocto associator. First, we define a velocity model to use. We go for a homogeneous velocity model. While a crude approximation in a subduction zone, it still gives good association. Check out the PyOcto documentation for details on velocity models and how to use 1D models.

In [ ]:
# Define homogeneous velocity model
velocity_model = pyocto.VelocityModel0D(
    p_velocity=6.0,
    s_velocity=6.0/1.75,
    tolerance=2.0,
    association_cutoff_distance=250,
)

Next, we create the associator. We use the `from_area()` function, as it will automatically select a local coordinate projection to use for the association. In addition, we set the overlap time (`time_before`), and pick-based quality control criteria.

In [ ]:
# Configure associator parameters - here we use geographic coordinates
# Hard coded for YH network
associator = pyocto.OctoAssociator.from_area(
    lat=(-39.5, -38.4),
    lon=(178, 179.5),
    zlim=(0, 200),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=5,
    n_p_and_s_picks=2,
)

Lastly, we convert the station information from the obspy inventory to the data frame required for PyOcto.

In [ ]:
stations = associator.inventory_to_df(inv)
stations

We now run the phase association. We use the `associate_seisbench()` function that directly takes the output of a SeisBench picking model as input. PyOcto offers further interfaces, for example, to input Pandas data frames. The association will take a moment (up to a few minutes on slower CPUs). The output are two dataframe:

- `events` contains a list of all events with their location and origin times
- `assignments` contains all picks and the event they are associated with

In [ ]:
# Run the associator
catalog, assignments = associator.associate_seisbench(picks, stations)

The events currently only contain locations in a local coordinate system. Let's transform it back to latitude and longitude before inspecting the events.

In [ ]:
associator.transform_events(catalog)

### Visualizing the catalog

Let's have a look at the list of events. Every event is listed with its time, local coordinates (x, y, z), the number of picks and the latitude, longitude and depth. Within the 12 hours of the example, we found over 400 event, on average one event every 100 seconds.

In [ ]:
catalog

We can also plot the catalog. Conveniently, we already have a local transverse mercator projection available, so no need for further thought in the plotting here. We use the scatter function and encode the depth of the events using color. The plot nicely resolves the intense shallow aftershock activity. It also shows the seismicity along the Slap (West-East dipping).

In [ ]:
# Initialize figure
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.set_aspect("equal")

# Show catalog events in km coordinates
cb = ax.scatter(catalog["x"], catalog["y"], c=catalog["z"], s=8, cmap="viridis")

# Add color bar for focal depth
cbar = fig.colorbar(cb)
cbar.ax.set_ylim(cbar.ax.get_ylim()[::-1])
cbar.set_label("Depth[km]")

# Add stations
ax.plot(stations["x"], stations["y"], "r^", ms=10, mew=1, mec="k")
ax.set_xlabel("Easting [km]")
ax.set_ylabel("Northing [km]")
plt.show()

As a last check, we manually inspect some events. The code block below selects a random event and plots the waveforms, together with the P (solid black lines) and S (dashed black lines). The x axis denotes the time, the y axis the distance between station and estimated event location. Therefore, we should see roughly a hyperbolic moveout. Run the cell a few times to see a few example events.

In [ ]:
# Choose random event in catalog
# event_idx = np.random.choice(catalog["idx"])
for event_idx in catalog["idx"]:
    
    # Extract picks and event info for this event
    event_picks = [picks[i] for i in assignments[assignments["event_idx"] == event_idx]["pick_idx"]]
    event = catalog[catalog["idx"] == event_idx].iloc[0]
    
    # Define station dictionary for ease of plotting
    station_dict = {station: (x, y) for station, x, y in zip(stations["id"], stations["x"], stations["y"])}
    
    # Extract window to include min and max pick times according to all picks
    first, last = min(pick.peak_time for pick in event_picks), max(pick.peak_time for pick in event_picks)
    
    sub = Stream()
    
    for station in np.unique([pick.trace_id for pick in event_picks]):
        sub.append(stream.select(station=station[3:-1], channel="?HZ")[0])
    
    # Initialize empty stream
    sub = sub.slice(first - 5, last + 5)
    
    # Pre-process the data: detrend and high-pass filter at 2 Hz
    sub = sub.copy()
    sub.detrend()
    sub.filter("highpass", freq=2)
    
    # Initialize figure
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111)
    
    # Plot the seismograms, ordered by total distance (hypocentral) from earthquake
    for i, trace in enumerate(sub):
        # Remove mean
        normed = trace.data - np.mean(trace.data)
        # Divide by max of absolute value
        normed = normed / np.max(np.abs(normed))
        # Extract x and y position of station
        station_x, station_y = station_dict[trace.id[:-4]]
        # Calculate euclidian hypocentral distance
        y = np.sqrt((station_x - event["x"]) ** 2 + (station_y - event["y"]) ** 2 + event["z"] ** 2)
        # Plot trace at corresponding distance
        ax.plot(trace.times(), 3 * normed + y)
        
    for pick in event_picks:
        # Extract x and y position of station
        station_x, station_y = station_dict[pick.trace_id]
        # # Calculate euclidian hypocentral distance
        y = np.sqrt((station_x - event["x"]) ** 2 + (station_y - event["y"]) ** 2 + event["z"] ** 2)
        # Calculate arrival time wrt window time
        x = pick.peak_time - trace.stats.starttime
        if pick.phase == "P":
            ls = '-'
        else:
            ls = '--'
        # Plot as vertical bars
        ax.plot([x, x], [y - 3, y + 3], 'k', ls=ls)
        
    # ax.set_ylim(0)
    ax.set_xlim(0, np.max(trace.times()))
    ax.set_ylabel("Hypocentral distance [km]")
    ax.set_xlabel("Time [s]")
    
    # Print out event information
    print("Event information")
    print(event)